# M4.2 — Beyond Pinecone Free Tier
## Cost Models, Alternatives, and Migration Strategy

**Learning Objectives:**
- Understand Pinecone pricing and cost drivers
- Compare alternatives: Weaviate, Qdrant, Elasticsearch
- Evaluate self-hosting vs managed services
- Apply decision frameworks by scale
- Identify hidden costs and troubleshooting scenarios

---
## Section 1: Pricing Reality Check

**Key Concept:** Vector databases excel at semantic search but cannot replace traditional databases.

### Pinecone Pricing Tiers

In [ ]:
import pandas as pd
import numpy as np

# Pinecone pricing structure (illustrative)
pricing_tiers = pd.DataFrame({
    'Tier': ['Free', 'Starter', 'Standard', 'Enterprise'],
    'Monthly Cost': ['$0', '$70', '$280+', 'Custom'],
    'Vector Capacity': ['100K', '100K-1M', '1M+', 'Unlimited'],
    'Pods': ['Serverless', '1 Standard', 'Multiple', 'Custom'],
    'Use Case': ['Dev/Test', 'Small Apps', 'Production', 'Large Scale']
})

print("# Expected: Pinecone Pricing Tiers")
print(pricing_tiers.to_string(index=False))

### Key Cost Drivers

1. **Number of Vectors:** Storage dominates cost at scale
2. **Embedding Dimensions:** Higher dims = more storage per vector
3. **Replicas:** Each replica = full pod cost for HA
4. **Query Volume:** Typically included up to 10M queries/month
5. **Indexes:** Multiple indexes multiply costs

In [ ]:
# Cost scaling by vector count
vector_counts = np.array([100_000, 500_000, 1_000_000, 5_000_000, 10_000_000])

# Illustrative monthly costs (USD)
def estimate_pinecone_cost(vectors):
    if vectors <= 100_000:
        return 0  # Free tier
    elif vectors <= 1_000_000:
        return 70  # Starter
    else:
        # Standard: ~$280 per million vectors
        return 280 * np.ceil(vectors / 1_000_000)

costs = [estimate_pinecone_cost(v) for v in vector_counts]

cost_table = pd.DataFrame({
    'Vectors': [f"{v:,}" for v in vector_counts],
    'Monthly Cost': [f"${c:.0f}" for c in costs],
    'Cost per 1K vectors': [f"${(c/(v/1000)):.4f}" if c > 0 else "$0" 
                            for v, c in zip(vector_counts, costs)]
})

print("# Expected: Cost Scaling by Vector Count")
print(cost_table.to_string(index=False))

### Reality Checks

**✅ Vector DBs ARE Good For:**
- Semantic similarity search ("find documents like this one")
- Unstructured data (text, images, audio embeddings)
- Sub-100ms queries across millions of vectors
- Horizontal scaling to billions of vectors

**❌ Vector DBs CANNOT:**
- Replace SQL databases (no ACID, foreign keys, complex joins)
- Guarantee result quality ("garbage embeddings in, garbage out")
- Provide value without data (empty index = useless)
- Handle traditional BI queries (aggregations, analytics)

In [ ]:
# Cost comparison: Vector DB vs Traditional DB
comparison = pd.DataFrame({
    'Feature': ['Storage Cost', 'Query Speed', 'Similarity Search', 'Transactions', 'Analytics'],
    'Vector DB': ['Higher', '<100ms', 'Native', 'Limited', 'Limited'],
    'Traditional DB': ['Lower', 'Variable', 'Poor/None', 'Full ACID', 'Excellent']
})

print("# Expected: Vector vs Traditional DB Comparison")
print(comparison.to_string(index=False))

**Key Insight:** Use vector databases alongside traditional databases, not as replacements. Most production systems need both.

---
## Section 2: Cost Estimator Walkthrough

Use the `m4_2_cost_models.py` module to calculate real-world cost scenarios.

In [ ]:
# Import the cost estimator
from m4_2_cost_models import PineconeCostEstimator, VectorDBComparison

print("✓ Cost models loaded")

### Scenario 1: Small App (500K vectors)

In [ ]:
# Small app: 500K vectors, 100K monthly queries
small_app = PineconeCostEstimator(vectors=500_000, monthly_queries=100_000)
result = small_app.estimate_monthly_cost()

print("# Expected: Small App Cost Breakdown")
print(f"Tier: {result['tier']}")
print(f"Storage Cost: ${result['storage_cost']:.2f}/mo")
print(f"Query Cost: ${result['query_cost']:.2f}/mo")
print(f"Total Monthly: ${result['total_monthly']:.2f}")
print(f"Cost per Vector: ${small_app.cost_per_vector():.6f}")
print(f"Annual Projection: ${small_app.annual_projection():.2f}")

### Scenario 2: Production App with HA (5M vectors, 2 replicas)

In [ ]:
# Production: 5M vectors, 2 replicas for HA
prod_app = PineconeCostEstimator(vectors=5_000_000, replicas=2, monthly_queries=1_000_000)
result = prod_app.estimate_monthly_cost()

print("# Expected: Production App Cost Breakdown")
print(f"Tier: {result['tier']}")
print(f"Storage Cost: ${result['storage_cost']:.2f}/mo")
print(f"Replica Cost: ${result['replica_cost']:.2f}/mo")
print(f"Query Cost: ${result['query_cost']:.2f}/mo")
print(f"Total Monthly: ${result['total_monthly']:.2f}")
print(f"Pods Needed: {result['pods_needed']}")
print(f"Annual Projection: ${prod_app.annual_projection():.2f}")

### Break-Even Analysis

At what vector count does Pinecone match alternative provider costs?

In [ ]:
# Break-even: Pinecone vs Qdrant ($100/mo alternative)
estimator = PineconeCostEstimator(vectors=1_000_000)
alternative_cost = 100  # Qdrant or Weaviate ~$100/mo

break_even_vectors = estimator.calculate_break_even(alternative_cost)

print("# Expected: Break-Even Analysis")
print(f"Alternative Cost: ${alternative_cost}/mo")
print(f"Break-even at: ~{break_even_vectors:,} vectors")
print(f"\nInterpretation:")
print(f"  Below {break_even_vectors:,} vectors: Pinecone may be cheaper")
print(f"  Above {break_even_vectors:,} vectors: Consider alternatives")

---
## Section 3: Provider Comparison

Compare Pinecone with open-source alternatives: **Weaviate**, **Qdrant**, **Elasticsearch**.

### Feature Comparison

In [ ]:
# Get feature comparison
features = VectorDBComparison.get_provider_features()

print("# Expected: Provider Feature Comparison")
print(features.to_string(index=False))

### Cost Comparison Across Scale

In [ ]:
# Compare costs across different scales
scenarios = [100_000, 500_000, 1_000_000, 5_000_000]
cost_comparison = VectorDBComparison.generate_comparison_table(scenarios)

print("# Expected: Cost Comparison Table")
print(cost_comparison.to_string(index=False))

### Provider Deep Dive

**Weaviate:**
- **Strength:** Native hybrid search (BM25 + semantic)
- **Flexibility:** Open-source with cloud option
- **Use Case:** Apps needing keyword + semantic search together
- **Pricing:** Sandbox free; ~$25/mo cloud; self-host free

**Qdrant:**
- **Strength:** Rust-based performance + memory efficiency
- **Filtering:** Excellent payload filtering capabilities
- **Use Case:** Cost-sensitive, high-performance deployments
- **Pricing:** Free 1GB; ~$25/mo cloud; self-host very efficient

**Elasticsearch:**
- **Strength:** Mature ecosystem, existing ES users
- **Hybrid:** Native BM25 + KNN vector search
- **Use Case:** Organizations already using ES for logs/search
- **Pricing:** ~$95/mo cloud starter; self-host needs expertise

**Key Insight:** Open-source options (Weaviate, Qdrant) offer **flexibility** to switch between managed and self-hosted as needs evolve.

---
## Section 4: When to Self-Host vs Managed

**The Trade-off:** Managed services cost more but save operational overhead. Self-hosting saves licensing but adds infrastructure + DevOps burden.

### Self-Hosting Infrastructure Costs

In [ ]:
# Estimate self-hosting costs on AWS (illustrative)
vector_scenarios = [500_000, 1_000_000, 5_000_000]

self_host_results = []
for vectors in vector_scenarios:
    infra = VectorDBComparison.self_host_infrastructure_estimate(vectors)
    self_host_results.append({
        'Vectors': f"{vectors:,}",
        'Compute': f"${infra['compute_monthly']:.2f}",
        'Storage': f"${infra['storage_monthly']:.2f}",
        'Network': f"${infra['network_monthly']:.2f}",
        'Total': f"${infra['total_infrastructure']:.2f}",
        'Storage GB': f"{infra['storage_gb']:.1f}"
    })

self_host_df = pd.DataFrame(self_host_results)

print("# Expected: Self-Host Infrastructure Costs (AWS)")
print(self_host_df.to_string(index=False))

### Decision Framework

**Choose Managed (Pinecone, Weaviate Cloud, Qdrant Cloud) when:**
- 🚀 **Speed to market** is critical (no ops setup)
- 👥 **Small/no DevOps team** available
- 📈 **Variable workload** (managed handles scaling)
- 💰 **Budget predictable** (trade $ for time)

**Choose Self-Hosted (Weaviate OSS, Qdrant OSS) when:**
- 💵 **Cost optimization** matters at scale (>5M vectors)
- 🔧 **DevOps expertise** in-house
- 📊 **Stable, predictable** workload
- 🔒 **Data sovereignty** requirements (on-prem/private cloud)

**Hidden Self-Host Costs:**
- DevOps time (~20-40 hrs/month for ops)
- Monitoring & alerting setup
- Backup/disaster recovery
- Security patching & updates

**Break-Even Rule of Thumb:**
Self-hosting becomes cost-effective around **5-10M vectors** if you have DevOps capacity.

---
## Section 5: Decision Cards by Scale

Recommendations based on vector count and use case.

### Scale 1: Prototype/MVP (<100K vectors)

**Recommendation:** Pinecone Free Tier or Qdrant Free
- **Cost:** $0/mo
- **Rationale:** Free tiers sufficient; defer vendor lock-in decisions
- **Watch out:** Plan migration before hitting 100K limit

**Example Use Cases:**
- Demo applications
- Personal projects
- Early-stage POCs

### Scale 2: Small Production (100K - 1M vectors)

**Recommendation:** Pinecone Starter or Weaviate/Qdrant Managed
- **Cost:** ~$25-70/mo
- **Rationale:** Managed simplicity, low ops burden
- **Consider:** Weaviate if hybrid search needed; Qdrant for cost efficiency

**Example Use Cases:**
- Small SaaS applications
- Internal company chatbots
- Niche document search

### Scale 3: Medium Production (1M - 10M vectors)

**Recommendation:** Evaluate Managed vs Self-Host
- **Managed Cost:** ~$280-2,800/mo (Pinecone) or ~$100-500/mo (Qdrant/Weaviate)
- **Self-Host Cost:** ~$150-300/mo infrastructure + DevOps time
- **Decision Factor:** Do you have DevOps capacity?

**If YES (have DevOps):** Consider self-hosting Qdrant/Weaviate
**If NO (no DevOps):** Stick with managed (prefer Qdrant/Weaviate for cost)

**Example Use Cases:**
- Mid-size SaaS platforms
- E-commerce product search
- Enterprise knowledge bases

### Scale 4: Large/Enterprise (>10M vectors)

**Recommendation:** Self-Host or Enterprise Contracts
- **Managed Cost:** $3,000+/mo (likely need custom pricing)
- **Self-Host Cost:** ~$500-1,500/mo infrastructure + ops
- **Break-even:** Self-hosting almost always cheaper at this scale

**Strategic Considerations:**
- Negotiate enterprise contracts with volume discounts
- Build in-house expertise for open-source solutions
- Consider hybrid: managed for dev/test, self-host for production

**Example Use Cases:**
- Large-scale content platforms
- Multi-tenant SaaS with millions of users
- National-scale search systems

### Decision Summary Table

In [ ]:
# Decision summary by scale
decision_summary = pd.DataFrame({
    'Scale': ['<100K', '100K-1M', '1M-10M', '>10M'],
    'Vectors': ['Prototype', 'Small Prod', 'Medium Prod', 'Enterprise'],
    'Recommendation': ['Free Tier', 'Managed', 'Evaluate Both', 'Self-Host'],
    'Typical Cost': ['$0', '$25-70', '$100-500', '$500-1,500+'],
    'Key Factor': ['Free limits', 'Simplicity', 'DevOps capacity', 'Cost optimization']
})

print("# Expected: Decision Summary by Scale")
print(decision_summary.to_string(index=False))

---
## Section 6: Troubleshooting & Hidden Costs

Real production failures and how to avoid them.

### Critical Production Failures

Based on real-world incidents documented in the source material.

In [ ]:
# Critical production failures and solutions
failures = pd.DataFrame({
    'Failure': [
        'Migration Data Loss',
        'Query Timeouts Under Load',
        'Memory Overflow',
        'Index Corruption',
        'API Rate Limiting'
    ],
    'Symptom': [
        '50%+ data loss during bulk transfer',
        'gRPC timeouts, connection pool exhaustion',
        'OOM errors with large vectors',
        'Corrupted indexes after crashes',
        'Cascading failures from rate limits'
    ],
    'Root Cause': [
        'Network timeouts without checkpoints',
        'High concurrent requests exceed limits',
        'Large vectors in constrained memory',
        'Writes interrupted by crashes',
        'Exceeding provider quotas'
    ],
    'Solution': [
        'Checkpoint-recovery logic after each batch',
        'Connection pooling + request queuing',
        'Memory profiling + right-size infrastructure',
        'Write-ahead logging + regular snapshots',
        'Throttling + exponential backoff + monitoring'
    ]
})

print("# Expected: Production Failure Patterns")
print(failures[['Failure', 'Solution']].to_string(index=False))

### Hidden Costs to Watch

**1. Vendor Lock-In Costs:**
- Migration effort if switching providers
- Re-architecture of application code
- Data export/import time and complexity
- **Mitigation:** Abstract vector DB behind an interface layer

**2. Latency Costs:**
- Cross-region network latency (can add 50-200ms)
- Cold-start delays in serverless environments
- **Mitigation:** Co-locate DB with application; use regional deployments

**3. Embedding Generation Costs:**
- OpenAI embeddings: ~$0.10 per 1M tokens
- For 1M documents (~500 tokens each): ~$50 in embedding costs
- **Hidden:** Often forgotten in TCO calculations

**4. Query Costs at Scale:**
- Most providers include 10M queries/month
- Beyond that: $5-10 per additional 1M queries
- High-traffic apps can exceed quotas quickly

**5. Testing & Staging Environments:**
- Need separate indexes for dev/test/staging
- Each environment multiplies costs
- **Mitigation:** Use smaller sample datasets for non-prod

### Key Takeaways

**Cost Drivers:**
1. Vector count dominates storage costs
2. Replicas double/triple costs for HA
3. Query costs kick in above 10M/month

**Provider Strategy:**
- **<1M vectors:** Managed services (Pinecone, Qdrant, Weaviate)
- **1-10M vectors:** Evaluate managed vs self-host based on DevOps capacity
- **>10M vectors:** Self-host almost always cheaper

**Risk Mitigation:**
- ✅ Implement checkpoint recovery for migrations
- ✅ Use connection pooling for high-load scenarios
- ✅ Monitor memory usage and right-size infrastructure
- ✅ Abstract DB behind interface to reduce lock-in
- ✅ Include embedding costs in TCO calculations

**Next Steps:**
1. Calculate your current/projected vector count
2. Use `m4_2_cost_models.py` to estimate costs
3. Compare providers using cost + feature matrices
4. Prototype with free tiers before committing
5. Plan migration strategy if scaling beyond free tier